In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import dbldatagen as dg
import dbldatagen.distributions as dist
from dbldatagen import FakerTextFactory, DataGenerator, fakerText
from pyspark.sql.types import LongType, FloatType, IntegerType, StringType, \
                              DoubleType, BooleanType, ShortType, \
                              TimestampType, DateType, DecimalType, \
                              ByteType, BinaryType, ArrayType, MapType, \
                              StructType, StructField

In [2]:
class DataGen:

    '''Class to Generate Synthetic Data'''

    def __init__(self, spark):
        self.spark = spark

    def transactionsDataGen(self, shuffle_partitions_requested = 10, partitions_requested = 10, data_rows = 10000):

        # partition parameters etc.
        self.spark.conf.set("spark.sql.shuffle.partitions", shuffle_partitions_requested)

        Dataspec = (DataGenerator(self.spark, rows=data_rows, partitions=partitions_requested)
                    .withColumn("p[1]", "float", minValue=300, maxValue=400, random=True)
                    .withColumn("p[2]", "float", minValue=100, maxValue=300, random=True)
                    .withColumn("n[1]", "float", minValue=0, maxValue=200, random=True)
                    )

        df = Dataspec.build()

        return df

In [3]:
import cml.data_v1 as cmldata

# Sample in-code customization of spark configurations
#from pyspark import SparkContext
#SparkContext.setSystemProperty('spark.executor.cores', '1')
#SparkContext.setSystemProperty('spark.executor.memory', '2g')

CONNECTION_NAME = "paul-aug26-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

myDG = DataGen(spark)

firstBatchDf = myDG.transactionsDataGen()

#Save DF as Iceberg Table
firstBatchDf.writeTo("DEFAULT.SALES_TABLE").using("iceberg").createOrReplace()
#pricesDf.write.format("json").mode("overwrite").save("s3a://goes-se-sandbox01/datalake/pdefusco/transactions/pricesDf.json")

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco


Spark Application Id:spark-application-1724886953627
